TODO:

看paper上使用的子集合部分的方法

In [9]:

import numpy as np
import itertools
import bisect
import math
import json
class Hypercube:                #超立方體
    '''
    A class to create a hypercube object which stores values on vertices
    and values on the edges between neighboring vertices
    '''    
    #輸入維度、(點鍵值)、(點值)
    def __init__(self, n_vertices, vertex_keys = None, vertex_values = None):   
        self.n_vertices = n_vertices
        self.v_num = 2**self.n_vertices
        self.V = [np.array([])] + all_subsets(n_vertices)   #所有子集包含空集，即所有點
        self.V_value = {str(v) : 0. for v in self.V}         #所有點值  先設為0
        self.E = []
        self.E_value = {}
        self.partial_gradient = {vertex : {} for vertex in range(n_vertices)}   #各個維度的partial gradient
        self.matrix = np.full((self.v_num,self.v_num),np.nan)                   #原始立方體對應到的矩陣
        self.partial_gradient_matrix = np.full((self.v_num,self.v_num),np.nan)  #對特定 feature的partial gradient
        self.vi_matrix = np.full((self.v_num,self.v_num),np.nan)                #v_i對應到的矩陣
        self.res_matrix = np.full((self.v_num,self.v_num),np.nan)               #residual的matrix
        self.sv = 0
        self.partial_gradient_norm = 0
    def set_vertex_values(self, vertex_values):         #設置點值
        for v in vertex_values:                         #用鍵值來做查找
            self.V_value[v] = vertex_values[v]
            
        # edge values are the differences between neighboring vertex values
        #self._calculate_edges()
        
    def _calculate_edges(self):                 #計算邊值
        
        # calculate the usual gradients: the difference between neighboring edges
        for i, v in enumerate(self.V):
            for _v in self.V[i+1:]:
                if self._vertices_form_a_valid_edge(v, _v):
                    self.E.append((v, _v))
                    self.E_value[str((v, _v))] = self.V_value[str(_v)] - self.V_value[str(v)]
        
        # calculate partial gradients
        for vertex in range(self.n_vertices):
            self.partial_gradient[vertex] = self.E_value.copy()
            for v, _v in self.E:
                is_relevant_edge_for_partial_gradient = (vertex in v and vertex not in _v) or (vertex in _v and vertex not in v)
                if not is_relevant_edge_for_partial_gradient:
                    self.partial_gradient[vertex][str((v, _v))] = 0.
            
    def _vertices_form_a_valid_edge(self, v, _v):       #檢查交集和是否相鄰
        # vertices are neighbors in a hypercube
        # if they differ by exactly one element
        differ_in_size_by_1 = (abs(len(v) - len(_v)) == 1)      #差距是1
    
        the_intersection = np.intersect1d(v, _v)                #兩個集合的交集
        #print(type(v[0]),type(_v[0]),type(the_intersection[0]))
        intersection_is_nonempty = len(the_intersection) > 0 or len(v)==0 or len(_v) == 0   #交集大於0或v,_v是空集
        is_intersection = False                         
        if len(the_intersection)>0:                              
            if len(the_intersection)==len(v) or len(the_intersection)==len(_v):
                is_intersection = True
        else:
            if len(v)==0 and len(_v)==1:
                is_intersection = True
       # print(is_intersection)
        return differ_in_size_by_1 and intersection_is_nonempty and is_intersection
    
    #create matrix for Hypercube
    def trans_to_matrix(self,feature_i):                #超立方體轉換成矩陣
        for i, v in enumerate(self.V):                  #對立方體上所有點
            for j,_v in enumerate(self.V[i+1:]):        
                if self._vertices_form_a_valid_edge(v, _v):     #是否相交
                    self.matrix[i][i+j+1] = self.V_value[str(_v)] - self.V_value[str(v)]    #獲得matrix
                    self.matrix[i+j+1][i] = self.V_value[str(v)] - self.V_value[str(_v)]
        
            
        self.partial_gradient_matrix = self.matrix.copy()       #把原始立方體複製下來，保留i方向
        self.my_list = []
        self.partial_gradient_vector = np.array([])
        self.weight_vector = np.array([])
        self.partial_gradient_norm = 0
        for j, v in enumerate(self.V):
            for k,_v in enumerate(self.V[j+1:]):
                if self._vertices_form_a_valid_edge(v, _v):     #如果是邊
                    is_relevant_edge_for_partial_gradient = (feature_i in v and feature_i not in _v) or (feature_i in _v and feature_i not in v)
                    if not is_relevant_edge_for_partial_gradient:           #如果 特徵i不在v 或_v，把那條邊設成0
                        self.partial_gradient_matrix[j][j+k+1] = 0.
                        self.partial_gradient_matrix[j+k+1][j] = 0.
                    else:
                        self.partial_gradient_norm += self.partial_gradient_matrix[j][j+k+1]**2
                        self.partial_gradient_vector = np.append(self.partial_gradient_vector,self.partial_gradient_matrix[j][j+k+1])      #比較向量1
                        subset_len = len(v)
                        weight = 1./(math.comb(self.n_vertices,subset_len)*(self.n_vertices-subset_len))
                        self.weight_vector = np.append(self.weight_vector,weight)
                        self.my_list.append((j,j+k+1))          
        #self.vi[feature_i] = self.shapley_residuals_in_matrix()
        #self.vi[i] = self.shapley_residuals_in_matrix
    
    #minimize the function (gradient_x - partial_gradient_i)^2 最小化l2_norm
    def shapley_residuals_in_matrix(self):
            derivative_i  = np.full((self.v_num,self.v_num),0.)      #計算微分後得到的方程式矩陣 A
            b_i = np.array([0.]*self.v_num)                          #得到Ax = b 的 b向量值
            for j  in range(self.v_num):                            #對矩陣的每個點
                for k in range(self.v_num):
                    if np.isnan(self.partial_gradient_matrix[j][k]):    #不用計算nan
                        continue
                    #elif j == 0 or k ==0:                               #如果是跟原點相鄰
                     #   derivative_i[j][j] += 1.                       #係數+1
                      #  b_i[j] += - self.partial_gradient_matrix[j][k]     #x_j-x_i-partial_gradient
                    else:                                               
                        derivative_i[j][j] += 1.                         
                        derivative_i[j][k] += -1.
                        b_i[j] += - self.partial_gradient_matrix[j][k]
            A = derivative_i[1:,1:]                                    #只要x_i for i!=0
            print(A)
            B = b_i[1:]                                                #保留b_i
            res = 0.
            #print('A_matrix:\n',A)
            #print('B_vector: ',B)
            A_inverse = np.linalg.inv(A)                               #算inverse matrix
            vi = np.insert(np.dot(A_inverse,B),0,0.)                    #vi  = b/A #開頭補0
            vi_V =  [np.array([])] + all_subsets(self.n_vertices)
            vi_V_value = {str(v) : 0. for v in vi_V} 
            res_dic = {}
            v_v = 0
            for k,v in enumerate(vi_V):               
                vi_V_value[str(v)] = vi[k]
            self.vi_vector = np.array([])                              #比較向量#2
            for i, v in enumerate(vi_V):
                for j,_v in enumerate(vi_V[i+1:]):
                    if self._vertices_form_a_valid_edge(v, _v):
                        self.vi_matrix[i][i+j+1] = vi_V_value[str(_v)] - vi_V_value[str(v)]
                        self.vi_matrix[i+j+1][i] = vi_V_value[str(v)] - vi_V_value[str(_v)]
                        self.res_matrix[i][i+j+1] = self.partial_gradient_matrix[i][i+j+1] - self.vi_matrix[i][i+j+1]
                        self.res_matrix[i+j+1][i] = self.partial_gradient_matrix[i+j+1][i] - self.vi_matrix[i+j+1][i]
                        res += (self.res_matrix[i][i+j+1])**2
                        v_v += (self.vi_matrix[i][i+j+1])**2
                        res_dic[str(v)+'->'+str(_v)] = self.res_matrix[i][i+j+1]
                        if (i,i+j+1) in self.my_list:
                            self.vi_vector = np.append(self.vi_vector,self.vi_matrix[i][i+j+1])     #比較向量#2
            #print(self.my_list)
            #print(self.partial_gradient_vector,self.vi_vector)
            print(self.partial_gradient_vector,self.vi_vector)
            vector_A = self.weight_vector*self.partial_gradient_vector
            vector_B = self.weight_vector*self.vi_vector
            print(vector_A,vector_B)
            cos_sim = self.cos_sim(vector_A,vector_B)
            print('cos_sim = ',cos_sim)
            self.sv += vi[-1]
            print('shapley_value: ',vi[-1],'residual sum: ',res)
            print(self.sv)
            #print('residuals_sum:',res,'shapley_value: ',vi)
            res =  (res/v_v)**0.5
            print('norm: ',res)
            return vi_V_value, res_dic,self.partial_gradient_norm
    
    def cos_sim(self,a,b):
        dot_product = np.dot(a, b)
        norm_a = np.linalg.norm(a)
        norm_b = np.linalg.norm(b)
        similarity = dot_product / (norm_a * norm_b)
        return similarity
            
def save_json(dic,feature_name):
    filename = f"dic_{feature_name}.json"
    with open(filename, 'w') as json_file:
        json.dump(dic, json_file)

def save_res_json(dic,feature_name):
    filename = f"dic_{feature_name}_res.json"
    with open(filename, 'w') as json_file:
        json.dump(dic, json_file)

    #def trans_partial_matrix(self):
        
####################
def all_subsets(n_elts):
    '''
        returns a list of 2^{n_elts} lists
        each a different subset of {1, 2,...,n_elts}
    '''
    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]
    res = {i : res[i] for i in range(n_elts)}
    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)
    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]

def get_residual(old_cube, new_cube, vertex):   #計算殘差
    '''
    returns: residual dictionary
        
        { edge : ▼_player_v[edge] - ▼v_player[edge] for edge in old_cube }
    '''
    assert set(old_cube.E_value.keys()) == set(new_cube.E_value.keys())     #判斷兩個字典中鍵值的組合是否相同。assert:
    res = {}
    for e in old_cube.E_value.keys():
        res[e] = old_cube.partial_gradient[vertex][e] - new_cube.E_value[e] #對應某特徵的邊相減 即gradient_i_v - gradient_v_i(殘差)
    return res
count = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
#記得添加number of vertex
def residual_norm(old_cube, vertex_values, vertex,num_features):     #old_cube是原本的SHAP得到的立方體
    '''
    old_cube: v, our game
    vertex: player
    vertex_values: v_player, proposed game
    
    assumes that the order of the values in vertex_values align with the order of the values in old_cube.V
    
    returns: || ▼_player_v - ▼v_player ||
    '''
    if count[vertex]==0 :
        count[vertex] += 1
    new_cube = Hypercube(num_features)
    new_cube.set_vertex_values({str(_vertex) : vertex_values[j] for j, _vertex in enumerate(old_cube.V)})   #將數值設定成0.5
    return np.sum([(r)**2 for r in get_residual(old_cube, new_cube, vertex).values()]), get_residual(old_cube, new_cube, vertex).values() #計算所有residual造成的影響加總
#改一下參數
def compute_residuals_v(old_cube,vertex_of_v_i_cube,_v,num_features):            #(instance cube,算出來的v_i cube,這個cube的指定feature)
    new_vertex =  np.append(np.array(0), vertex_of_v_i_cube)
    new_c = Hypercube(num_features)
    coalitions = [np.array([])] + all_subsets(num_features)
    b = {}
    for i, coalition in enumerate(coalitions):
        b[str(coalition)] = new_vertex[i]
    new_c.set_vertex_values(b)
    res = get_residual(old_cube,new_c,_v)
    return(res.values())




In [10]:
feature_num = 3
feature_n = ['X','Y','Z']
import numpy as np 
import itertools
def all_subsets(n_elts):
    '''
        returns a list of 2^{n_elts} lists
        each a different subset of {1, 2,...,n_elts}
    '''
    res = [np.array(list(itertools.combinations(set(range(n_elts)), i))) for i in range(n_elts)]
    res = {i : res[i] for i in range(n_elts)}
    res[n_elts] = np.array([i for i in range(n_elts)]).reshape(1,-1)
    return [res[i][j] for i in range(1,n_elts+1) for j in range(res[i].shape[0])]
def f_x(x_list):
    return (x_list[0]+ 2*x_list[1]*x_list[2] )
pers = [np.array([])] + all_subsets(feature_num)   #所有帶值subset
a={}
test_value = []
for i,per in enumerate(pers):
    x_l = [0]*feature_num
    for j in range(feature_num):
        if j in per:
            x_l[j] = 1
    test_value.append(f_x(x_l))
    a[str(per)] = test_value[i]

#print(a)
test_cube = Hypercube(feature_num)
test_cube.set_vertex_values(a)
res_vector = []
partial_norm_va = []
#for k  in range(3):
    #b = compute_residuals_v(test_cube,gradient_vi[k],k,3)
    #round_b = [ round(k,2) for k in b ]
    #print(round_b)
    #res_r.append(sum([abs(r) for r in b]))
#print(test_cube.partial_gradient[1].values())
#print(test_cube.E_value.values())
#print(res_r)
for i  in range(feature_num):
    test_cube.trans_to_matrix(i)
    res_n = test_cube.shapley_residuals_in_matrix()
    #print(res_n[1])
    save_json(res_n[0],feature_name=feature_n[i])
    save_res_json(res_n[1],feature_name=feature_n[i])
    partial_norm_va.append(res_n[2])
    res_vector.append(res_n[1])
print('here')
print(partial_norm_va,res_vector)
    #print('residuals_sum/shapley_value: ',res_n)
#print('=========')
#print(test_cube.matrix)
#print(test_cube.partial_gradient_matrix)
#print(test_cube.vi[1])

[[ 3.  0.  0. -1. -1.  0.  0.]
 [ 0.  3.  0. -1.  0. -1.  0.]
 [ 0.  0.  3.  0. -1. -1.  0.]
 [-1. -1.  0.  3.  0.  0. -1.]
 [-1.  0. -1.  0.  3.  0. -1.]
 [ 0. -1. -1.  0.  0.  3. -1.]
 [ 0.  0.  0. -1. -1. -1.  3.]]
[1. 1. 1. 1.] [1. 1. 1. 1.]
[0.33333333 0.16666667 0.16666667 0.33333333] [0.33333333 0.16666667 0.16666667 0.33333333]
cos_sim =  0.9999999999999998
shapley_value:  1.0000000000000002 residual sum:  2.973635834133892e-31
1.0000000000000002
norm:  2.7265526925652344e-16
[[ 3.  0.  0. -1. -1.  0.  0.]
 [ 0.  3.  0. -1.  0. -1.  0.]
 [ 0.  0.  3.  0. -1. -1.  0.]
 [-1. -1.  0.  3.  0.  0. -1.]
 [-1.  0. -1.  0.  3.  0. -1.]
 [ 0. -1. -1.  0.  0.  3. -1.]
 [ 0.  0.  0. -1. -1. -1.  3.]]
[0. 0. 2. 2.] [0.5 0.5 1.5 1.5]
[0.         0.         0.33333333 0.66666667] [0.16666667 0.08333333 0.25       0.5       ]
cos_sim =  0.9486832980505138
shapley_value:  0.9999999999999999 residual sum:  2.0
2.0
norm:  0.5773502691896257
[[ 3.  0.  0. -1. -1.  0.  0.]
 [ 0.  3.  0. -1.  0. -1

In [15]:

    
def compare_dependency(A,pair):
    resi_vec = []
    p_n =0
    for fea in pair:
        p_n += A[fea]
        with open('dic_'+feature_n[fea]+'_res.json') as f:
            data = json.load(f)
            resi_vec.append(data)
    results = []
    r_no = 0
    print(resi_vec)
    for key in resi_vec[0]:
        if key in resi_vec[1]:
            results.append(resi_vec[0][key]+resi_vec[1][key])
            
    for result in results:
        r_no += result**2
    
    return (r_no/p_n)**0.5

compare_dependency(partial_norm_va,[0,2])

    

[{'[]->[0]': 0.0, '[]->[1]': -1.942890293094024e-16, '[]->[2]': 1.1102230246251565e-16, '[0]->[0 1]': 0.0, '[0]->[0 2]': 0.0, '[1]->[0 1]': 2.220446049250313e-16, '[1]->[1 2]': 1.942890293094024e-16, '[2]->[0 2]': 0.0, '[2]->[1 2]': -1.1102230246251565e-16, '[0 1]->[0 1 2]': -2.220446049250313e-16, '[0 2]->[0 1 2]': -2.220446049250313e-16, '[1 2]->[0 1 2]': -2.220446049250313e-16}, {'[]->[0]': 2.220446049250313e-16, '[]->[1]': 0.5, '[]->[2]': -0.5, '[0]->[0 1]': 0.5000000000000002, '[0]->[0 2]': -0.5000000000000002, '[1]->[0 1]': 4.440892098500626e-16, '[1]->[1 2]': 0.5, '[2]->[0 2]': 0.0, '[2]->[1 2]': -0.5, '[0 1]->[0 1 2]': 0.4999999999999998, '[0 2]->[0 1 2]': -0.4999999999999998, '[1 2]->[0 1 2]': 2.220446049250313e-16}]


0.408248290463863

In [12]:
import numpy as np

a = np.array([0., 0., 0.25, 0., 0.25, 0., 0.25, 0.75])
b = np.array([0.1875, 0.0625, 0.1875, 0.0625, 0.1875, 0.0625, 0.1875, 0.5625])

# 计算点积
dot_product = np.dot(a, b)

# 计算向量长度
norm_a = np.linalg.norm(a)
norm_b = np.linalg.norm(b)

# 计算余弦相似度
cosine_similarity = dot_product / (norm_a * norm_b)

angle_radians = np.arccos(cosine_similarity)

angle_degrees = np.degrees(angle_radians)

print("Angle (degrees):", angle_degrees)

Angle (degrees): 18.434948822922017
